In [ ]:
%load_ext autoreload
%autoreload 2


from liftout.structures import Sample, load_experiment


import os 
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# path = r"C:\Users\Admin\Github\autoliftout\liftout\log\dm-embryo-05_2022-08-29.11-57-03AM" # 6
path = r"C:\Users\Admin\Github\autoliftout\liftout\log\dm-embryo-06_2022-08-30.03-26-27PM" # 3

sample = load_experiment(path)
sample.__to_dataframe__()

### Log Parsing (new)

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from copy import deepcopy
from dataclasses import dataclass

from pathlib import Path

from liftout.tools.data import calculate_statistics_dataframe, AutoLiftoutStatistics, calculate_aggregated_statistics


paths = [
    r"C:\Users\Admin\Github\autoliftout\liftout\log\dm-embryo-05_2022-08-29.11-57-03AM",
    r"C:\Users\Admin\Github\autoliftout\liftout\log\dm-embryo-06_2022-08-30.03-26-27PM"
]





In [ ]:
df = calculate_aggregated_statistics(paths)
display(df)

In [ ]:
# specimen current
microscope.state.specimen_current.value

## System Settings
Set / Restore system settings


In [ ]:
%load_ext autoreload
%autoreload 2
from fibsem import utils, calibration
from fibsem.structures import BeamSystemSettings, SystemSettings, BeamType

from autoscript_sdb_microscope_client import SdbMicroscopeClient

from pprint import pprint 

In [ ]:
microscope, settings = utils.setup_session()

In [ ]:
eb_settings = calibration.get_current_beam_system_state(microscope, BeamType.ELECTRON)
ib_settings = calibration.get_current_beam_system_state(microscope, BeamType.ION)
pprint(eb_settings)
pprint(ib_settings)

In [ ]:
from dataclasses import asdict
# check if settings are different
for i, (a, b) in enumerate(list(zip(asdict(eb_settings).values(), asdict(settings.system.electron).values()))):

    if a != b:
        k = list(asdict(eb_settings).keys())[i]
        print("not equal: ", k, a, b)

## History Bug

In [5]:
%load_ext autoreload
%autoreload 2

from liftout.structures import create_experiment, Lamella, AutoLiftoutStage
from liftout.autoliftout import update_sample_lamella_data
from fibsem import utils
from pprint import pprint
from liftout import autoliftout


microscope, settings = utils.setup_session()
os.makedirs("test", exist_ok=True)
sample = create_experiment("test-history", "test")


2022-10-05 19:08:25,367 — root — INFO — connect_to_microscope:28 — Microscope client connecting to [10.0.0.1]
2022-10-05 19:08:25,372 — root — INFO — connect_to_microscope:31 — Microscope client connected to [10.0.0.1]
2022-10-05 19:08:25,373 — root — INFO — setup_session:241 — Finished setup for session: demo_2022-10-05.07-08-25PM


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Client connecting to [10.0.0.1:7520]...
Client connected to [10.0.0.1:7520]



        Lamella 1 (01-active-horse). 
        Lamella Coordinates: StagePosition(), 
        Landing Coordinates: StagePosition(), 
        Current Stage: AutoLiftoutStage.Setup,
        History: 0 stages completed ([]).
        
Sample: 
        Path: test\test-history-2022-10-05.07-08-25PM
        State: None
        Lamella: 1
        


In [23]:
lamella = Lamella(sample.path, 1)
sample = update_sample_lamella_data(sample, lamella)


stages = [stage for stage in AutoLiftoutStage][2:]

for stage in stages:
    print(stage)
    pprint(sample)
    pprint(sample.positions[1])

    lamella.current_state.stage = stage
    sample = autoliftout.end_of_stage_update(microscope, sample, lamella, )
    
    pprint(sample)
    pprint(sample.positions[1])
    pprint([state.stage.name for state in sample.positions[1].history])
    print("-"*50)

2022-10-05 19:19:52,801 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.MillTrench | FINISHED
2022-10-05 19:19:52,828 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.MillJCut | FINISHED
2022-10-05 19:19:52,851 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.Liftout | FINISHED
2022-10-05 19:19:52,878 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.Landing | FINISHED
2022-10-05 19:19:52,908 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.Reset | FINISHED
2022-10-05 19:19:52,936 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.Thinning | FINISHED
2022-10-05 19:19:52,969 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.Polishing | FINISHED


AutoLiftoutStage.MillTrench
Sample: 
        Path: test\test-history-2022-10-05.07-08-25PM
        State: None
        Lamella: 1
        

        Lamella 1 (01-in-moth). 
        Lamella Coordinates: StagePosition(), 
        Landing Coordinates: StagePosition(), 
        Current Stage: AutoLiftoutStage.Setup,
        History: 0 stages completed ([]).
        
Sample: 
        Path: test\test-history-2022-10-05.07-08-25PM
        State: None
        Lamella: 1
        

        Lamella 1 (01-in-moth). 
        Lamella Coordinates: StagePosition(), 
        Landing Coordinates: StagePosition(), 
        Current Stage: AutoLiftoutStage.MillTrench,
        History: 1 stages completed (['MillTrench']).
        
['MillTrench']
--------------------------------------------------
AutoLiftoutStage.MillJCut
Sample: 
        Path: test\test-history-2022-10-05.07-08-25PM
        State: None
        Lamella: 1
        

        Lamella 1 (01-in-moth). 
        Lamella Coordinates: StagePosition()

2022-10-05 19:19:53,010 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.Finished | FINISHED
2022-10-05 19:19:53,050 — root — INFO — end_of_stage_update:980 — 01-in-moth | AutoLiftoutStage.Failure | FINISHED


Sample: 
        Path: test\test-history-2022-10-05.07-08-25PM
        State: None
        Lamella: 1
        

        Lamella 1 (01-in-moth). 
        Lamella Coordinates: StagePosition(), 
        Landing Coordinates: StagePosition(), 
        Current Stage: AutoLiftoutStage.Polishing,
        History: 7 stages completed (['MillTrench', 'MillJCut', 'Liftout', 'Landing', 'Reset', 'Thinning', 'Polishing']).
        
['MillTrench',
 'MillJCut',
 'Liftout',
 'Landing',
 'Reset',
 'Thinning',
 'Polishing']
--------------------------------------------------
AutoLiftoutStage.Finished
Sample: 
        Path: test\test-history-2022-10-05.07-08-25PM
        State: None
        Lamella: 1
        

        Lamella 1 (01-in-moth). 
        Lamella Coordinates: StagePosition(), 
        Landing Coordinates: StagePosition(), 
        Current Stage: AutoLiftoutStage.Polishing,
        History: 7 stages completed (['MillTrench', 'MillJCut', 'Liftout', 'Landing', 'Reset', 'Thinning', 'Polishing']).
 

[AutoLiftoutState(stage=<AutoLiftoutStage.MillTrench: 1>, microscope_state=MicroscopeState(timestamp=1664957992.78096, absolute_position=StagePosition(x=-0.015512917, y=0.052503333, z=0.023157247, t=0, r=0.8553286, coordinate_system=Raw), eb_settings=BeamSettings(beam_type=<BeamType.ELECTRON: 1>, working_distance=0.00495, beam_current=5e-11, hfw=4.9899091826437944e-05, resolution='1536x1024', dwell_time=1e-06), ib_settings=BeamSettings(beam_type=<BeamType.ION: 2>, working_distance=0.0165, beam_current=3e-11, hfw=0.00015, resolution='1536x1024', dwell_time=5e-07)), start_timestamp=None, end_timestamp=1664957992.792929),
 AutoLiftoutState(stage=<AutoLiftoutStage.MillJCut: 2>, microscope_state=MicroscopeState(timestamp=1664957992.802902, absolute_position=StagePosition(x=-0.015512917, y=0.052503333, z=0.023157247, t=0, r=0.8553286, coordinate_system=Raw), eb_settings=BeamSettings(beam_type=<BeamType.ELECTRON: 1>, working_distance=0.00495, beam_current=5e-11, hfw=4.9899091826437944e-05, re